In [21]:
num_of_words = 10

In [22]:
from konlpy.tag import Okt
import pickle
import re

try:
    with open('crawling/news.pickle', 'rb') as f:
        articles = pickle.load(f)
except FileNotFoundError:
    with open('../crawling/news.pickle', 'rb') as f:
        articles = pickle.load(f)

bodies = articles['body'].to_list()

okt = Okt()

sentences = []
for body in bodies:
    for sentence in body.split('.'):
        if not re.match('.*[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9 ].*', sentence):
            sentences.append(sentence)

data = [(okt.nouns(sentence), okt.morphs(sentence)) for sentence in sentences]
data = [(keywords, sentence) for keywords, sentence in data
        if 0 < len(keywords) <= num_of_words]

def remove_overlaps(keywords):
    result = []
    for keyword in keywords:
        if keyword not in result:
            result.append(keyword)
    return result

data = [(remove_overlaps(keywords), sentence) for keywords, sentence in data]